<a href="https://colab.research.google.com/github/aokhader/OcelotBot/blob/main/Tina_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("hi")

hi


In [ ]:
#This sample Ocelot bot uses gpt-2 as the base model, and the dailydialog dataset from Hugging Face as training data (https://huggingface.co/datasets/roskoN/dailydialog)

# Install necessary libraries
!pip install transformers datasets torch

# Import required libraries
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset

# Step 1: Load the Dataset
# Load the dataset from Hugging Face
dataset = load_dataset("roskoN/dailydialog")

# Preview the dataset structure
print(dataset['train'][0])



INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.3 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


0000.parquet:   0%|          | 0.00/3.67M [00:00<?, ?B/s]

full/validation/0000.parquet:   0%|          | 0.00/340k [00:00<?, ?B/s]

full/test/0000.parquet:   0%|          | 0.00/337k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11118 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'id': 'a438b751ab9997cdb35f07bfe3dfb010b96365f4762d77f87e5f41290ff61c3d_0', 'acts': [3, 4, 2, 2, 2, 3, 4, 1, 3, 4], 'emotions': [0, 0, 0, 0, 0, 0, 4, 4, 4, 4], 'utterances': ['Say , Jim , how about going for a few beers after dinner ?', 'You know that is tempting but is really not good for our fitness .', 'What do you mean ? It will help us to relax .', "Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?", "I guess you are right.But what shall we do ? I don't feel like sitting at home .", 'I suggest a walk over to the gym where we can play singsong and meet some of our friends .', "That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them .", 'Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too .', "Good.Let ' s go now .", 'All right .']}


In [ ]:
# Step 2: Tokenization
# Load GPT-2 tokenizer and model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token (GPT-2 doesn't have one by default)
tokenizer.pad_token = tokenizer.eos_token

# Function to tokenize the dataset
def tokenize_function(examples):
    # Extract the 'dialog' field from examples
    dialogs = examples['dialog'] # Corrected the key to 'dialog'

    # Tokenize each utterance in the dialog and concatenate them
    input_ids = []
    attention_mask = []
    for dialog in dialogs:
        dialog_ids = []
        dialog_mask = []
        for utterance in dialog:
            tokens = tokenizer(utterance, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
            dialog_ids.extend(tokens['input_ids'].tolist())
            dialog_mask.extend(tokens['attention_mask'].tolist())
        input_ids.append(dialog_ids)
        attention_mask.append(dialog_mask)

    return {'input_ids': input_ids, 'attention_mask': attention_mask}

# Apply the tokenization function to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Convert to PyTorch tensors
# (This step might be redundant if your tokenizer already returns tensors)
#tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Step 3: Prepare for Training
# Split into training and validation datasets
train_dataset = tokenized_datasets['train']
eval_dataset = tokenized_datasets['validation']

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    push_to_hub=False,
    logging_dir='./logs',
    logging_steps=10
)

# Step 4: Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Step 5: Fine-tuning the GPT-2 Model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_gpt2_dailydialog")
tokenizer.save_pretrained("./fine_tuned_gpt2_dailydialog")

# Step 6: Inference Function
def generate_response(history, max_length=150):
    input_text = " ".join(history)
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(inputs, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example conversation generation
conversation_history = ["Hello! How are you today?", "I'm doing well, thank you! How about you?"]
response = generate_response(conversation_history)
print(response)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/11118 [00:00<?, ? examples/s]

KeyError: 'dialog'

In [ ]:
# Step 2: Tokenization
# Load GPT-2 tokenizer and model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token (GPT-2 doesn't have one by default)
tokenizer.pad_token = tokenizer.eos_token

# Function to tokenize the dataset
#def tokenize_function(examples):
    #return tokenizer(examples['utterances'], padding="max_length", truncation=True, max_length=128)

def tokenize_function(examples):
    # Flatten the list of utterances in case it's a nested list of lists
    flattened_utterances = [item for sublist in examples['utterances'] for item in (sublist if isinstance(sublist, list) else [sublist])]

    # Join the flattened list into a single string
    dialog_text = " ".join(flattened_utterances)

    # Tokenize the concatenated dialog text
    return tokenizer(dialog_text, padding="max_length", truncation=True, max_length=128)

# Apply the tokenization function to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Convert to PyTorch tensors
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Step 3: Prepare for Training
# Split into training and validation datasets
train_dataset = tokenized_datasets['train']
eval_dataset = tokenized_datasets['validation']

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    push_to_hub=False,
    logging_dir='./logs',
    logging_steps=10
)

# Step 4: Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Step 5: Fine-tuning the GPT-2 Model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_gpt2_dailydialog")
tokenizer.save_pretrained("./fine_tuned_gpt2_dailydialog")

# Step 6: Inference Function
def generate_response(history, max_length=150):
    input_text = " ".join(history)
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(inputs, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example conversation generation
conversation_history = ["Hello! How are you today?", "I'm doing well, thank you! How about you?"]
response = generate_response(conversation_history)
print(response)

Map:   0%|          | 0/11118 [00:00<?, ? examples/s]

ArrowInvalid: Column 4 named input_ids expected length 1000 but got length 128